## All code written by Amirsadra Mohseni
Supplemental material provided by UC Riverside

Requires pandas and numpy packages

## Modules

In [3]:
import pandas as pd
import numpy as np
from time import perf_counter
from collections import Counter

## KNN Classifier

In [4]:
def knn_classify(test_data, training_data, training_labels, k=1):
    # Distance b/w test instance and all training data instances
    dist = np.linalg.norm(test_data - training_data, axis=1)

    distance_label_pairs = dict(zip(dist, training_labels)) # Assign distances with labels (distance : label)
    distance_label_pairs = sorted(distance_label_pairs.items()) # sort based on keys (distances)

    k_nearest = distance_label_pairs[0:k] # Only need the first k pairs

    # Return the most common label in distance:label pairs
    return Counter([pair[1] for pair in k_nearest]).most_common(1)[0][0]

## Leave-One-Out Cross-Validation

In [5]:
def leave_one_out_cross_validation(data, all_labels, list_of_features, feature_to_add=None, k=1):
    all_accuracy = []
    features = []
    features = list(list_of_features) # Make a copy before append
    
    if (feature_to_add != None):
        features.append(feature_to_add)
    
    for i in range(0, len(data)):
        # Pick out the i-th row and the desired feature columns
        test_instance = data.iloc[i, features]
        test_label = all_labels[i]
        
        # Drop the i-th row, select all rows (except the i-th which is dropped) and the desired feature columns
        train_instances = data.drop(i).iloc[:, features]
        train_labels = list(all_labels.drop(i))

        prediction = knn_classify(test_instance, train_instances, train_labels, k)
        
        if (prediction == test_label):
            all_accuracy.append(1) # Correct pred
        else:
            all_accuracy.append(0) # Incorrect pred
    
    return sum(all_accuracy)/len(all_accuracy) # Mean

## Feature Search

In [6]:
def feature_search(data, all_labels, k, verbose=1):
    set_of_features = []
    subsets = []
    subsets_accuracy = []
    
    print("Beginning search...")
    
    for i in range(0, len(data.columns)):
        if verbose: print("Level " + str(i) + " of the search tree")
        else: print("[" + str(i) + "]/[" + str(len(data.columns) - 1) + "] Working...")
        
        this_level_feature = 0
        best_accuracy_so_far = 0
        
        for feature_to_add in range(0, len(data.columns)):
            
            if (feature_to_add in set_of_features):
                # only consider adding if not already added
                continue
            
            if verbose: print("--Considering adding feature " + str(feature_to_add))
            accuracy = leave_one_out_cross_validation(data, all_labels, set_of_features, feature_to_add, k)
            if verbose: print("----Accuracy of feature " + str(feature_to_add) + ": " + str(round(accuracy, 2)))
            
            if (accuracy > best_accuracy_so_far):
                best_accuracy_so_far = accuracy
                this_level_feature = feature_to_add
                
        set_of_features.append(this_level_feature)
        
        subsets.append(list(set_of_features))
        subsets_accuracy.append(best_accuracy_so_far)
        
        if verbose: print("On level " + str(i) + " added feature " + str(this_level_feature) + " to current set\n\n")
    
    most_accuracy = max(subsets_accuracy)
    most_accurate_subset = subsets[subsets_accuracy.index(most_accuracy)]
    
    print("Most accurate subset is " + str(most_accurate_subset) + " with an accuracy of " + str(most_accuracy * 100) + "%")
    #print("All subsets: " + str(subsets))
    #print("All accuracies: " + str(subsets_accuracy))
    
    return most_accurate_subset, most_accuracy, subsets, subsets_accuracy

## Main

In [7]:
def main():
    print("Welcome to Amir's Feature Selection Algorithm")
    file_name = input("Type in the name of the file to test or Enter nothing for small80.txt: ") or "small80.txt"
    
    num_cols = len(pd.read_csv(file_name, nrows=1, delim_whitespace=True).columns) - 1
    col_names=['CLASS']

    for i in range(num_cols):
        col_names.append("F" + str(i))

    df = pd.read_csv(file_name, delim_whitespace=True, names=col_names)
    all_vals = df.drop("CLASS", axis=1)
    all_labels = df["CLASS"]
    
    print("\nThis dataset has " + str(num_cols) + " features (not including the CLASS attribute), with " + str(len(df)) + " instances.")
    norm_yes_no = int(input("Normalize data? 1 for yes, 0 for no. Enter nothing for default yes (z-score norm)") or 1)
    
    # z-score normalize
    if norm_yes_no:
        print("Please wait while I z-score normalize the data... ", end="")
        all_vals = (all_vals - all_vals.mean()) / all_vals.std(ddof=0)
        print("Done.")
    
    print("Using Forward Selection algorithm")
    
    k = int(input("\nWhich k would you like to use with KNN? Enter nothing for default 1: ") or 1)
    
    verbose = int(input("\nShow detailed search tree info? Enter 1 for yes, and 0 for no. Enter nothing for default 1: ") or 1)
    
    acc = leave_one_out_cross_validation(all_vals, all_labels, [x for x in range(0, num_cols)], None, k=k)
    
    print("\nRunning nearest neighbor with all " + str(num_cols) +
          " features, using “leaving-one-out” evaluation, I get an accuracy of " + str(acc) + "\n")
    
    t1_start = perf_counter()
    #-----------------TIME-----------------
    feature_search(all_vals, all_labels, k, verbose=verbose)
    #-----------------TIME-----------------
    t1_stop = perf_counter()
    s = (t1_stop - t1_start)
    print("--- %s seconds ---" % s)

In [ ]:
main()

Welcome to Amir's Feature Selection Algorithm


Type in the name of the file to test or Enter nothing for small80.txt:  



This dataset has 10 features (not including the CLASS attribute), with 100 instances.


Normalize data? 1 for yes, 0 for no. Enter nothing for default yes (z-score norm) 


Please wait while I z-score normalize the data... Done.
Using Forward Selection algorithm



Which k would you like to use with KNN? Enter nothing for default 1:  

Show detailed search tree info? Enter 1 for yes, and 0 for no. Enter nothing for default 1:  



Running nearest neighbor with all 10 features, using “leaving-one-out” evaluation, I get an accuracy of 0.65

Beginning search...
Level 0 of the search tree
--Considering adding feature 0
----Accuracy of feature 0: 0.57
--Considering adding feature 1
----Accuracy of feature 1: 0.54
--Considering adding feature 2
----Accuracy of feature 2: 0.68
--Considering adding feature 3
----Accuracy of feature 3: 0.65
--Considering adding feature 4
----Accuracy of feature 4: 0.75
--Considering adding feature 5
----Accuracy of feature 5: 0.61
--Considering adding feature 6
----Accuracy of feature 6: 0.62
--Considering adding feature 7
